In [1]:
from kiteconnect import KiteConnect
import pandas as pd
from selenium import webdriver
import time
import os
from datetime import date
pd.options.mode.chained_assignment = None

In [4]:
token_path = 'api_key.txt'
key_secret = open(token_path,'r').read().split()   #gives a list object
kite = KiteConnect(api_key = key_secret[0])
#virtually start the chromedriver -  don't make the window pop up
service = webdriver.chrome.service.Service('C:/Users/MAHE/.wdm/drivers/chromedriver/83.0.4103.39/win32/chromedriver.exe')
service.start()
#don't render the browser, save some RAM
options = webdriver.ChromeOptions()
options.add_argument('--headless')
#just aggregating all the capabilities like headless, memoryless etc
options = options.to_capabilities()
#start the driver, pass the options 
driver = webdriver.Remote(service.service_url,options)
driver.get(kite.login_url())
driver.implicitly_wait(2)
#fint the element in the webpage by the HTML path
username = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[1]/input')
password = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[2]/input')
username.send_keys(key_secret[2])
password.send_keys(key_secret[3])
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[4]/button').click()
pin = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[2]/div/input')
pin.send_keys(key_secret[4])
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[3]/button').click()
time.sleep(2)
request_token = driver.current_url.split('=')[1].split('&action')[0]
with open('request_token.txt','w') as file:
    file.write(request_token)
driver.quit()

#Use request_token and get_access_token
request_token = open('request_token.txt','r').read().split()
key_secret = open('api_key.txt','r').read().split()
kite = KiteConnect(api_key = key_secret[0])
data = kite.generate_session(request_token[0],api_secret = key_secret[1])
with open('access_token.txt','w') as file:
    file.write(data['access_token'])
    
# Set access token and you're all set!!!!!!!!!!   
kite.set_access_token(open('access_token.txt','r').read().split()[0])

In [60]:
stock_list = pd.read_excel('nif_top100.xlsx')

In [61]:
stock_list

,Sr. No.,Symbol,Company Name,Market Capitalisation \n(In lakhs)
0,1,RELIANCE,Reliance Industries Limited,7.060358e+07
1,2,TCS,Tata Consultancy Services Limited,6.852230e+07
2,3,HINDUNILVR,Hindustan Unilever Limited,4.975841e+07
3,4,HDFCBANK,HDFC Bank Limited,4.724823e+07
4,5,HDFC,Housing Development Finance Corporation Limited,2.823571e+07
...,...,...,...,...
995,996,MBAPL,Madhya Bharat Agro Products Limited,1.440368e+04
996,997,SANGAMIND,Sangam (India) Limited,1.438887e+04
997,998,CHEMFAB,Chemfab Alkalis Limited,1.405719e+04
998,999,GOKEX,Gokaldas Exports Limited,1.393813e+04


In [62]:
instrument_dump = kite.instruments("NSE")
instrument_df = pd.DataFrame(instrument_dump)

In [63]:
symbols = list(np.intersect1d(stock_list.Symbol, np.intersect1d(stock_list.Symbol, instrument_df.tradingsymbol)))

In [65]:
symbols

963

In [66]:
def instrumentLookup(symbols):
    global instrument_df
    
    tokens = []
    for i in range(len(symbols)):
        tokens.append(instrument_df[instrument_df.tradingsymbol == symbols[i]].instrument_token.values[0])
        
    return tokens

In [67]:
tokens = instrumentLookup(symbols)

In [74]:
from sklearn.linear_model import LinearRegression 
import math

adjusted_slope = []
for name in tokens:
    #if i ==1:
        
        #break
    data = pd.DataFrame(kite.historical_data(name,'2020-04-01','2020-06-22','day'))#start = date(2020,1,1
    
    x_data = np.array(data['close'])
    y_data = np.arange(1,len(data['close'])+1)
    log_x_data = np.log(x_data)
    reg = LinearRegression().fit(np.array(y_data).reshape(-1,1),np.array(log_x_data).reshape(-1,1))  
    c_m = np.corrcoef(np.array(y_data).reshape(-1,1).T,np.array(log_x_data).reshape(-1,1).T)
    
    c_xy = c_m[0,1]
    
    r_squared = c_xy**2
    

    adjusted_slope.append((math.pow((math.exp(reg.coef_)),250)-1)*(r_squared))
   

In [75]:
import operator
index, value = max(enumerate(adjusted_slope), key=operator.itemgetter(1))

In [76]:
value

38142.37498191802

In [77]:
symbols[index]

'ALOKINDS'

In [78]:
dictionary = dict(zip(symbols,adjusted_slope))
from collections import OrderedDict
d_ascending = OrderedDict(sorted(dictionary.items(), key=lambda kv: kv[1],reverse = True))

In [79]:
d_ascending

OrderedDict([('ALOKINDS', 38142.37498191802),
             ('IDEA', 381.6442848792501),
             ('GTLINFRA', 219.07043005724114),
             ('BORORENEW', 67.63656213436806),
             ('ADANIGREEN', 47.816025888795025),
             ('DEN', 42.18643021220941),
             ('SHREDIGCEM', 39.70522835252094),
             ('HATHWAY', 37.32207612628702),
             ('JUBILANT', 34.37823782106332),
             ('ASTEC', 33.04470243004411),
             ('TV18BRDCST', 31.57473546727811),
             ('HCG', 26.380609660465566),
             ('NATHBIOGEN', 23.739281672467442),
             ('JINDALSTEL', 23.535872620670666),
             ('NETWORK18', 21.74805428979687),
             ('HINDOILEXP', 21.3302802004284),
             ('JPPOWER', 20.92396440423893),
             ('TATACOMM', 18.666522850946244),
             ('EIDPARRY', 17.471211299016947),
             ('AARTIDRUGS', 17.057126983712468),
             ('DHANUKA', 15.659803109990975),
             ('VSTTILLERS', 15